In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd

csv_path = 'PLANILLA-CARGA-MONEDAS.csv'

def get_date():
    today = datetime.today()
    yesterday = today - timedelta(days=1)
    if today.weekday() == 0: #Lunes -> Viernes
        date = today - timedelta(days=3)
    else:
        date = yesterday

    return date

def get_quotes_arca(date):
    url = 'https://serviciosweb.afip.gob.ar/aduana/cotizacionesMaria/formulario.asp'

    payload = {
        'dia': date.strftime('%d'),
        'mes': date.strftime('%m'),
        'anio': date.strftime('%Y'),
        'consultarConstancia.x': 0,
        'consultarConstancia.y': 0
    }

    headers = {'User-Agent': 'Mozilla/5.0'}
    session = requests.Session()
    resp = session.post(url, data=payload, headers=headers)
    soup = BeautifulSoup(resp.text, 'html.parser')

    tablas = soup.find_all('table', class_='contenido')

    if not tablas:
        print("⚠️ No encontré la tabla de cotizaciones, puede que no haya datos para la fecha.")
        return pd.DataFrame()

    tabla = tablas[0]

    monedas_filtrar = ['DOLAR ESTADOUNIDENSE', 'ESTERLINA', 'EURO', 'SUIZO', 'BOLIVIANO', 'AUSTRALIANO']
    monedas_comprador = ['DOLAR ESTADOUNIDENSE','EURO']   
    datos = []
    
    for fila in tabla.find_all('tr')[1:]:  # salto cabecera
        columnas = fila.find_all('td')
        nombre_moneda = columnas[0].get_text(strip=True).upper()
        if any(m in nombre_moneda for m in monedas_filtrar):
                moneda = columnas[0].get_text(strip=True)
                fecha = columnas[1].get_text(strip=True)
                vendedor = columnas[2].get_text(strip=True).replace('.',',')#VENDEDOR
                datos.append([moneda,0, vendedor, fecha,'A'])
        if any(m in nombre_moneda for m in monedas_comprador):
                moneda = columnas[0].get_text(strip=True)
                fecha = columnas[1].get_text(strip=True)
                comprador = columnas[3].get_text(strip=True).replace('.',',')#COMPRADOR
                datos.append([moneda,0, comprador, fecha,'A'])


    df = pd.DataFrame(datos, columns=['U.M', '0', 'TC', 'FECHA','A'])

    return df




,U.M,0,TC,FECHA,A
0,DOL\r\n -\r...,0,1315,8/21/2025,A
1,DOL\r\n -\r...,0,1306,8/21/2025,A
2,021\r\n -\r...,0,"1767,36",8/21/2025,A
3,060\r\n -\r...,0,"1528,6875",8/21/2025,A
4,060\r\n -\r...,0,"1514,96",8/21/2025,A
5,009\r\n -\r...,0,"1627,791711",8/21/2025,A
6,026\r\n -\r...,0,"846,071",8/21/2025,A
7,031\r\n -\r...,0,"190,648786",8/21/2025,A


In [ ]:
from datetime import datetime, timedelta

today = datetime.today()-timedelta(days=1)
print((today))

TypeError: unsupported operand type(s) for -: 'str' and 'datetime.timedelta'